In [1]:
import pandas as pd
news=pd.read_json("news.json",lines=True)

In [2]:
#df_test=pd.read_csv("test.csv")

In [3]:
news

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [4]:
news['Text']=news['headline']+news['short_description']

In [5]:
news['source']=news.groupby('category').ngroup()

In [6]:
group=news.groupby(['source', 'category']).size().reset_index()

In [7]:
group=group[['source','category']]

In [8]:
group

,source,category
0,0,ARTS
1,1,ARTS & CULTURE
2,2,BLACK VOICES
3,3,BUSINESS
4,4,COLLEGE
5,5,COMEDY
6,6,CRIME
7,7,CULTURE & ARTS
8,8,DIVORCE
9,9,EDUCATION


In [9]:
y=news.source
x=news.drop('source',axis=1)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
test=y_test.to_frame()
train=y_train.to_frame()
df_test=pd.merge(x_test, test, left_index=True, right_index=True)
df_train=pd.merge(x_train, train, left_index=True, right_index=True)

In [11]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
stopWords = set(stopwords.words('english'))

def textcleaner_stem(Text): # Need Change here
    
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', Text.lower()) # Need Change here
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        if w not in stopWords:
            if len(w)>2:
                clean.append(ps.stem(w))
    return ' '.join(clean)

In [12]:
df_train['clean_text_stem'] = df_train.Text.apply(lambda x: textcleaner_stem(x))
df_test['clean_text_stem'] = df_test.Text.apply(lambda x: textcleaner_stem(x))

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stopWords = set(stopwords.words('english'))

def textcleaner_lemmas(Text): # Need Change here
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', Text.lower()) # Need Change here
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        if w not in stopWords:
            if len(w)>2:
                clean.append(lemmatizer.lemmatize(w))
    return ' '.join(clean)

In [14]:
df_train['clean_text_lemma'] = df_train.Text.apply(lambda x: textcleaner_lemmas(x))
df_test['clean_text_lemma'] = df_test.Text.apply(lambda x: textcleaner_lemmas(x))

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=6, strip_accents='ascii', analyzer='word', lowercase=True,
                             ngram_range=(1,2))

x_train_stem = vectorizer.fit_transform(df_train['clean_text_stem'])
y_train_stem = df_train['source']
x_test_stem = vectorizer.transform(df_test['clean_text_stem'])
y_test_stem = df_test['source']
features_train = vectorizer.get_feature_names()

In [16]:
from sklearn.naive_bayes import MultinomialNB

#Initialize and fit
nb = MultinomialNB()
nb.fit(x_train_stem, y_train_stem)

# Apply to testing data
y_pred_stem = nb.predict(x_test_stem)


# Showing model performance
print("Accuracy is: %0.3f" % nb.score(x_test_stem, y_test_stem))

Accuracy is: 0.451


In [17]:
vectorizer = TfidfVectorizer(min_df=6, strip_accents='ascii', analyzer='word', lowercase=True,
                             ngram_range=(1,2))

x_train_lemma = vectorizer.fit_transform(df_train['clean_text_lemma'])
y_train_lemma = df_train['source']
x_test_lemma = vectorizer.transform(df_test['clean_text_lemma'])
y_test_lemma = df_test['source']
features_train = vectorizer.get_feature_names()
len(features_train)

58836

In [18]:
from sklearn.naive_bayes import MultinomialNB


#Initialize and fit
nb = MultinomialNB()
nb.fit(x_train_lemma, y_train_lemma)

# Apply to testing data
y_pred_lemma = nb.predict(x_test_lemma)


# Showing model performance
print("Accuracy is: %0.3f" % nb.score(x_test_lemma, y_test_lemma))

Accuracy is: 0.458


In [34]:
from sklearn.linear_model import SGDClassifier


#Initialize and fit
nb = SGDClassifier(loss="hinge", penalty="l2", max_iter=250)
nb.fit(x_train_lemma, y_train_lemma)

# Apply to testing data
y_pred_lemma = nb.predict(x_test_lemma)


# Showing model performance
print("Accuracy is: %0.3f" % nb.score(x_test_lemma, y_test_lemma))

Accuracy is: 0.581


In [18]:
X_train = x_train_lemma
Y_train = y_train_lemma
X_test = x_test_lemma 
Y_test = y_test_lemma

In [ ]:
from sklearn.svm import SVC


# Create instance and fit
sv = SVC(kernel='linear')
sv.fit(X_train, Y_train)

# Apply to testing data
y_pred = sv.predict(X_test)

# Showing model performance
print("Accuracy is: %0.3f" % sv.score(X_test, Y_test))

In [16]:
from sklearn.linear_model import LogisticRegression


lr = LogisticRegression()
lr.fit(X_train, Y_train)

y_pred = lr.predict(X_test)


# Showing model performance
print("Accuracy is: %0.3f" % lr.score(X_test, Y_test))

Accuracy is: 0.906


C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [21]:
from sklearn import ensemble


gbc = ensemble.GradientBoostingClassifier()
gbc.fit(X_train, Y_train)

pred = gbc.predict(X_test)

# Showing model performance
print("Accuracy is: %0.3f" % gbc.score(X_test, Y_test))

Accuracy is: 0.511


In [20]:
from sklearn import ensemble


rfc = ensemble.RandomForestClassifier()
rfc.fit(X_train, Y_train)

y_pred = rfc.predict(X_test)


# Showing model performance
print("Accuracy is: %0.3f" % rfc.score(X_test, Y_test))

Accuracy is: 0.545
